`Grouper`permite crear `groupby`con instrucciones mas precisas. Tiene una utilidad importante en el manejo de fechas, por ejemplo, en la agrupación de datos generando intervalos de fechas con la posibilidad de indicar el inicio, el final y el paso en diferentes unidades: minutos, horas, días, etc.

In [1]:
import pandas as pd
import numpy as np
import sys

from sklearn import datasets

In [2]:
print(f"""Creado: 2021-01-18
Modificado: {pd.Timestamp.now()}
Testeado en:

    - Python {sys.version}
    - Versión pandas: {pd.__version__}
""")

Creado: 2021-01-18
Modificado: 2021-06-05 22:10:35.546948
Testeado en:

    - Python 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]
    - Versión pandas: 1.1.3



# Pandas Grouper

## Grouper sobre datos categóricos

In [3]:
df = pd.DataFrame({
    'Nombre': ['Paul', 'Alicia', 'Murakami', 'Paul', 'Alicia'],
    'Cantidad': [23, 2, 12, 34, 6]
})

df

,Nombre,Cantidad
0,Paul,23
1,Alicia,2
2,Murakami,12
3,Paul,34
4,Alicia,6


In [4]:
group = pd.Grouper(key='Nombre')
df.groupby(group).sum()

,Cantidad
Nombre,
Paul,57
Alicia,8
Murakami,12


La variable `group` es entonces un objeto de tipo `Grouper` sobre la columna (axis=0) Nombre (key='Nombre') y sin ordenamiento (Sort=False).

In [5]:
group

Grouper(key='Nombre', axis=0, sort=False)

Estos parámetros puedes ser extraidos de la variable `group` como atributos del objeto `Grouper`.

In [6]:
group.key

'Nombre'

In [7]:
group.axis

0

In [8]:
group.sort

False

## Intervalos con fechas

Una de las funciones utilizadas en pandas para crear intervalos de fechas es `date_range`. Haciendo uso de esta se crea un intervalo dada la fecha inicial y la fecha final con una frecuencia de 20 min:

In [9]:
inicio = '2020-05-02 4:24:00'
fin = '2020-05-02 8:17:00'
rango = pd.date_range(inicio, fin, freq='20min')
df = pd.DataFrame(rango, columns=['Fecha'])

df

,Fecha
0,2020-05-02 04:24:00
1,2020-05-02 04:44:00
2,2020-05-02 05:04:00
3,2020-05-02 05:24:00
4,2020-05-02 05:44:00
5,2020-05-02 06:04:00
6,2020-05-02 06:24:00
7,2020-05-02 06:44:00
8,2020-05-02 07:04:00
9,2020-05-02 07:24:00


Ahora se pretende realizar un conteo, por hora, de los registros del intervalo creado anteriormente. Para esto se crea un `Grouper` indicando el nombre de la columna *Fecha* y se especifica la frecuencia en horas usando `freq='H'`.

In [10]:
df.groupby(pd.Grouper(key='Fecha', freq='H')).size()

Fecha
2020-05-02 04:00:00    2
2020-05-02 05:00:00    3
2020-05-02 06:00:00    3
2020-05-02 07:00:00    3
2020-05-02 08:00:00    1
Freq: H, dtype: int64

Se realiza un ejercicio análogo con una frecuencia de 30 minutos.

In [11]:
df.groupby(pd.Grouper(key='Fecha', freq='30min')).size()

Fecha
2020-05-02 04:00:00    1
2020-05-02 04:30:00    1
2020-05-02 05:00:00    2
2020-05-02 05:30:00    1
2020-05-02 06:00:00    2
2020-05-02 06:30:00    1
2020-05-02 07:00:00    2
2020-05-02 07:30:00    1
2020-05-02 08:00:00    1
Freq: 30T, dtype: int64

El argumento `offset` permite agregarle un tiempo al origen de la serie y el origen viene definido en el argumento `origin`, que por defecto es `start_day`, en el cual se considera el primer día a media noche de la serie que se está agrupando. Consideremos el siguiente ejemplo sobre el dataframe `df`: 

- `origin='start'` indica que se tendrá como referencia el primer valor de la serie de tiempo, en este caso corresponde a *2020-05-02 4:24:00*,
- `offset='2min'` con lo que se le agregan estos dos minutos al primer valor de la serie de tiempo lo que da como resultado *2020-05-02 4:26:00*,
- `freq='30min'` indica la frecuencia de la serie.

Noten que al aumentar 2 minutos sobre el valor inicial de la serie esl primer valor (*2020-05-02 4:24:00*) es menor que el dado por el offset *2020-05-02 4:26:00* por lo que debe toamrse un valor previo utilizando la frecuencia. En resumen:

- Primer valor de la serie original: 2020-05-02 4:24:00
- Valor mas el offset: 2020-05-02 4:24:00 + 2min = 2020-05-02 4:26:00
- Valor inicial en la serie agrupada: 2020-05-02 4:26:00 - 30min = 2020-05-02 03:56:00

In [12]:
group = pd.Grouper(key='Fecha', freq='30min', offset='2min', origin='start')
df.groupby(group).size()

Fecha
2020-05-02 03:56:00    1
2020-05-02 04:26:00    1
2020-05-02 04:56:00    2
2020-05-02 05:26:00    1
2020-05-02 05:56:00    2
2020-05-02 06:26:00    1
2020-05-02 06:56:00    2
2020-05-02 07:26:00    1
2020-05-02 07:56:00    1
Freq: 30T, dtype: int64

Hay una diferencia al crear un `Grouper` sobre datos categóricos  y sobre fechas. Para este último se crea un objeto de tipo `TimeGrouper` que contiene aún mas atributos que pueden ser accedidos de la misma forma que sobre el objeto de tipo `Grouper`.

In [13]:
group

TimeGrouper(key='Fecha', freq=<30 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', origin='start', offset=Timedelta('0 days 00:02:00'))

In [14]:
group.freq

<30 * Minutes>

In [15]:
group.offset

Timedelta('0 days 00:02:00')

## Tiempo de ejecución con el groupby

In [16]:
df = datasets.load_breast_cancer(as_frame=True)['frame']
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Utilizar el `Grouper`o solo el `groupby` no tiene realmente una diferencia considerable en el tiempo de ejecución:

In [17]:
group = pd.Grouper(key='target')
%timeit df.groupby(group).sum()
%timeit df.groupby('target').sum()

987 µs ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
994 µs ± 3.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
